# Lab 5: Direct Lake Framing

## Introduction

This lab explores the critical concept of **framing** in Microsoft Fabric Direct Lake models. Framing is the automatic process by which Direct Lake models track and synchronize data changes in the underlying lakehouse tables. Understanding framing behavior is essential for optimizing refresh performance and ensuring data consistency in real-time analytics scenarios.

## Lab Overview

**Learning Objectives:**
- Understand automatic vs. manual framing mechanisms
- Observe how data modifications trigger framing events
- Learn to control framing behavior for optimal performance
- Master manual reframing techniques for large-scale updates

**Key Concepts:**
- **Automatic Framing**: Real-time synchronization of data changes
- **Manual Framing**: Controlled refresh for batch operations
- **Frame Tracking**: Monitoring data modification history
- **Performance Optimization**: Balancing refresh frequency with query performance

**Architecture Overview:**
```
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│   Data Sources  │───▶│  Lakehouse       │───▶│  Direct Lake    │
│                 │    │  (Delta Tables)  │    │  Model          │
└─────────────────┘    └──────────────────┘    └─────────────────┘
                               │                         │
                               ▼                         ▼
                       ┌──────────────────┐    ┌─────────────────┐
                       │  Framing Engine  │───▶│  Power BI       │
                       │  (Change Track)  │    │  Reports        │
                       └──────────────────┘    └─────────────────┘
```

**Prerequisites:** Lab 4 completion (fallback behavior understanding)

## 1. Install Semantic Link Labs Python Library
Install the Semantic Link Labs library for advanced Direct Lake operations and framing analysis.

In [ ]:
# Install semantic-link-labs library for Direct Lake operations
%pip install -q --disable-pip-version-check semantic-link-labs

## 2. Install Python Libraries
Set up required libraries and configure lakehouse/semantic model variables for framing operations.

In [ ]:
# Import libraries for Direct Lake operations and data manipulation
import sempy_labs as labs
import time
import pandas

# Configure lakehouse and semantic model names
LakehouseName = "AdventureWorks"
lakehouses = labs.list_lakehouses()["Lakehouse Name"]
for l in lakehouses:
    if l.startswith("Adventure"):
        LakehouseName = l

# Set up workspace and lakehouse identifiers
SemanticModelName = f"{LakehouseName}_model"
workspaceId = notebookutils.lakehouse.getWithProperties(LakehouseName)["workspaceId"]
lakehouseId = notebookutils.lakehouse.getWithProperties(LakehouseName)["id"]

## 3. Connect to Workspace and Validate Setup
Verify workspace connection and confirm lakehouse/semantic model availability for framing operations.

In [ ]:
# Validate workspace connection and display setup information
print(f"Workspace ID: {workspaceId}")
print(f"Lakehouse Name: {LakehouseName}")
print(f"Lakehouse ID: {lakehouseId}")
print(f"Semantic Model Name: {SemanticModelName}")

# Verify lakehouse exists and is accessible
try:
    lakehouse_info = labs.list_lakehouses()
    print(f"\nAvailable Lakehouses: {len(lakehouse_info)} found")
    print("Setup validation complete!")
except Exception as e:
    print(f"Error accessing lakehouse: {e}")

## 4. Load Report Generation Libraries
Import specialized helper functions for Power BI report creation and framing analysis.

In [ ]:
# Import specialized helper functions for Power BI operations and framing analysis
from sempy_labs._helper_functions import (
    resolve_report_id,
    format_dax_object_name,
    resolve_dataset_from_report,
    _conv_b64,
    _extract_json,
    _add_part,
    _decode_b64,
    resolve_workspace_name_and_id,
    _update_dataframe_datatypes,
    _base_api,
    _create_dataframe,
)

## 5. Create Simple Power BI Report
Generate a basic Power BI report to visualize data changes and monitor framing behavior in real-time.

In [ ]:
report_name="Simple Report"

pbi_report:dict = {}
pbi_report['config'] = """{
        "version": "5.37",
        "themeCollection": {},
        "activeSectionIndex": 0,
        "linguisticSchemaSyncVersion": 0,
        "objects": {
            "outspacePane": [
                {
                    "properties": {
                        "expanded": {
                            "expr": {
                                "Literal": {
                                    "Value": "false"
                                }
                            }
                        }
                    }
                }
            ]
        }
    }"""
pbi_report['layoutOptimization']=0
pbi_report['resourcePackages'] = [{'resourcePackage': {'disabled': False, 'items': [{'name': 'CY24SU10', 'path': 'BaseThemes/CY24SU10.json', 'type': 202}], 'name': 'SharedResources', 'type': 2}}]
pbi_report['sections'] = [
    {'config': '{}', 
    'displayName': 'Page 1', 
    'displayOption': 1, 
    'filters': '[]', 
    'height': 300.0, 
    'width': 600.0,
    'name': 'a4c1ed461808909ae820', 
    'visualContainers':
        [
            {'config': '''{
                        "name": "Matrix",
                        "layouts": [
                            {
                                "id": 0,
                                "position": {
                                    "x": 310,
                                    "y": 30,
                                    "z": 1000,
                                    "width": 253,
                                    "height": 202
                                }
                            }
                        ],
                        "singleVisual": {
                            "visualType": "tableEx",
                            "projections": {
                                "Values": [
                                    {
                                        "queryRef": "DimDate.Month"
                                    },
                                    {
                                        "queryRef": "Sum(FactInternetSales.SalesAmount)"
                                    },
                                    {
                                        "queryRef": "Sum(FactInternetSales.DiscountAmount)"
                                    }
                                ]
                            },
                            "prototypeQuery": {
                                "Version": 2,
                                "From": [
                                    {
                                        "Name": "d",
                                        "Entity": "DimDate",
                                        "Type": 0
                                    },
                                    {
                                        "Name": "f",
                                        "Entity": "FactInternetSales",
                                        "Type": 0
                                    }
                                ],
                                "Select": [
                                    {
                                        "Column": {
                                            "Expression": {
                                                "SourceRef": {
                                                    "Source": "d"
                                                }
                                            },
                                            "Property": "Month"
                                        },
                                        "Name": "DimDate.Month",
                                        "NativeReferenceName": "Month"
                                    },
                                    {
                                        "Aggregation": {
                                            "Expression": {
                                                "Column": {
                                                    "Expression": {
                                                        "SourceRef": {
                                                            "Source": "f"
                                                        }
                                                    },
                                                    "Property": "SalesAmount"
                                                }
                                            },
                                            "Function": 0
                                        },
                                        "Name": "Sum(FactInternetSales.SalesAmount)",
                                        "NativeReferenceName": "Sum of SalesAmount"
                                    },
                                    {
                                        "Aggregation": {
                                            "Expression": {
                                                "Column": {
                                                    "Expression": {
                                                        "SourceRef": {
                                                            "Source": "f"
                                                        }
                                                    },
                                                    "Property": "DiscountAmount"
                                                }
                                            },
                                            "Function": 0
                                        },
                                        "Name": "Sum(FactInternetSales.DiscountAmount)",
                                        "NativeReferenceName": "Discount"
                                    }
                                ]
                            },
                            "columnProperties": {
                                "Sum(FactInternetSales.DiscountAmount)": {
                                    "displayName": "Discount"
                                }
                            },
                            "drillFilterOtherVisuals": true,
                            "vcObjects": {
                                "dropShadow": [
                                    {
                                        "properties": {
                                            "show": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "true"
                                                    }
                                                }
                                            }
                                        }
                                    }
                                ]
                            }
                        }
                    }''', 'filters': '[]', 'height': 202.46, 'width': 215.11, 'x': 319.67, 'y': 30.63, 'z': 1.0
                },
            {'config': 
                    '''{
                        "name":"Card",
                        "layouts":[
                            {
                            "id":0,
                            "position":{"x":10,"y":30,"z":0,"width":238,"height":201}}
                            ],
                        "singleVisual":{"visualType":"card","projections":{"Values":[{"queryRef":"FactInternetSales.Count of Sales"}]},


                            "prototypeQuery": {
                                "Version": 2,
                                "From": [
                                    {
                                        "Name": "f",
                                        "Entity": "FactInternetSales",
                                        "Type": 0
                                    }
                                ],
                                "Select": [
                                    {
                                        "Measure": {
                                            "Expression": {
                                                "SourceRef": {
                                                    "Source": "f"
                                                }
                                            },
                                            "Property": "Count of Sales"
                                        },
                                        "Name": "FactInternetSales.Count of Sales",
                                        "NativeReferenceName": "Count of Sales"
                                    }
                                ],
                                "OrderBy": [
                                    {
                                        "Direction": 2,
                                        "Expression": {
                                            "Measure": {
                                                "Expression": {
                                                    "SourceRef": {
                                                        "Source": "f"
                                                    }
                                                },
                                                "Property": "Count of Sales"
                                            }
                                        }
                                    }
                                ]
                            },



                            "drillFilterOtherVisuals":true,
                            "hasDefaultSort":true,                           
                            "objects": {
                                "labels": [
                                    {
                                        "properties": {
                                            "fontSize": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "20D"
                                                    }
                                                }
                                            },
                                            "labelDisplayUnits": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "1D"
                                                    }
                                                }
                                            }
                                        }
                                    }
                                ]
                            },
                            "vcObjects": {
                                "dropShadow": [
                                    {
                                        "properties": {
                                            "show": {
                                                "expr": {
                                                    "Literal": {
                                                        "Value": "true"
                                                    }
                                                }
                                            }
                                        }
                                    }
                                ]
                            }
                            }
                        }''',
                 'filters': '[]', 
                 'height': 201.5, 
                 'width': 265.43, 
                 'x': 270.03, 
                 'y': 30.12, 
                 'z': 1000.0
            }
        ]
    }]

labs.report.create_report_from_reportjson(report=report_name , dataset="AdventureWorks_model" , report_json = pbi_report)
report_id = resolve_report_id(report_name)

from powerbiclient import Report
report = Report(group_id=None, report_id=report_id)
report.set_size(400,700)
report

## 6. Show Lakehouse Tables
Display all available tables in the lakehouse to understand the data structure for framing operations.

In [ ]:
# Display all available tables in the lakehouse for framing analysis
display(labs.list_lakehouse_tables(lakehouse=LakehouseName))

## 7. Show Relationships for AdventureWorks Model
Examine the semantic model relationships that affect framing behavior across related tables.

In [ ]:
# Display semantic model relationships to understand framing dependencies
display(labs.list_relationships(SemanticModelName))

## 8. Show History Details for DimDate
Analyze the modification history of the DimDate table to understand baseline framing behavior.

In [ ]:
# Analyze DimDate table modification history to establish baseline
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="DimDate"))

## 9. Show History Details for FactInternetSales
Examine the transaction history of the fact table to establish a baseline for framing monitoring.

In [ ]:
# Examine FactInternetSales table history to understand current state
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales"))

## 10. Insert Data to FactInternetSales Using Append
Perform data insertions to trigger automatic framing and observe real-time synchronization behavior.

In [ ]:
# Get one day of data from existing table
from pyspark.sql.functions import lit, min, max ,count
df1 = spark.read.load(f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")

# Show Min, MAX and Count of rows
# df1.agg(
#     min("OrderDateKey").alias("min_OrderDateKey") ,
#     max("OrderDateKey").alias("max_OrderDateKey") ,
#     count("*").alias("count_rows")
#     ).show()


# Create a filtered dataframe to update and then append back onto the original table
df2 = df1.filter("OrderDateKey='20221204'")
df2 = df2.withColumn("OrderDateKey",lit(20050630))


df2.write.mode("append").save(f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales"))
report

In [ ]:
df1 = spark.read.load(f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")

# Show Min, MAX and Count of rows
df1.agg(
        min("OrderDateKey").alias("min_OrderDateKey") ,
        max("OrderDateKey").alias("max_OrderDateKey") ,
        count("*").alias("count_rows")
        ).show()


## 11. Load FactInternetSales into Variable
Cache the fact table data in memory for efficient bulk data modification operations.

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")


## 12. Delete Some Rows from FactInternetSales
Execute delete operations to observe how data removal triggers framing events and affects model performance.

In [ ]:
deltaTable.delete("OrderDateKey = '20050701'")
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales"))
report

## 13. Update Some Rows in FactInternetSales
Perform update operations to demonstrate how data modifications trigger automatic framing mechanisms.

In [ ]:
deltaTable.update(
    condition= col("OrderDateKey")=='20220218',
    set = { "DiscountAmount":"1"}
)
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales"))
report

## 14. Turn OFF Auto Framing
Disable automatic framing to control when the model synchronizes with data changes for performance optimization.

## 15. Insert More Data to FactInternetSales Using Append
Perform additional data insertions while auto framing is disabled to demonstrate manual control.

In [ ]:
# Create filtered dataframe for appending new data while auto framing is disabled
df2 = df1.filter("OrderDateKey='20221204'")
df2 = df2.withColumn("OrderDateKey",lit(20050629))

# Append data to table and monitor framing behavior
df2.write.mode("append").save(f"abfss://{workspaceId}@onelake.dfs.fabric.microsoft.com/{lakehouseId}/Tables/FactInternetSales")
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales"))
report

## 16. Update Some Rows from FactInternetSales
Execute updates with auto framing disabled to observe how changes accumulate without immediate synchronization.

In [ ]:
# Update specific rows while auto framing is disabled
deltaTable.update(
    condition= col("OrderDateKey")=='20220218',
    set = { "DiscountAmount":"2"}
)
time.sleep(4)
# Check history to see accumulated changes without automatic framing
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales"))
report

## 17. Delete Some Rows from FactInternetSales
Perform deletions while framing is disabled to demonstrate batch operation efficiency.

In [ ]:
deltaTable.delete("OrderDateKey = '20050702'")
time.sleep(4)
display(labs.delta_analyzer_history(lakehouse=LakehouseName, table_name="FactInternetSales"))
report

## 18. Reframe Model to Update Changes
Manually trigger a reframe operation to synchronize all accumulated changes with the Direct Lake model.

In [ ]:
# Manually trigger reframe operation to synchronize all accumulated changes
labs.refresh_semantic_model(SemanticModelName)

## 19. Stop the Spark Session
Clean up resources by properly terminating the Spark session used for framing operations.

In [ ]:
# Clean up resources by stopping the Spark session
mssparkutils.session.stop()

## Lab Summary

### 🎯 Key Achievements

**Framing Mastery Completed:**
- ✅ **Automatic Framing**: Observed real-time data synchronization behavior
- ✅ **Manual Control**: Learned to disable and control framing for performance
- ✅ **Batch Operations**: Demonstrated efficient bulk data processing techniques
- ✅ **Performance Optimization**: Balanced refresh frequency with query performance

### 📊 Framing Behavior Analysis

**Automatic Mode Benefits:**
- Real-time data consistency across Power BI reports
- Immediate reflection of data changes in analytics
- Seamless user experience for operational reporting

**Manual Mode Advantages:**
- Optimized performance for large-scale data operations
- Reduced resource consumption during batch processing
- Controlled refresh timing for scheduled updates

### 🔧 Best Practices Learned

**Framing Strategy Guidelines:**
1. **Use Automatic Framing** for real-time operational dashboards
2. **Disable Auto Framing** during large ETL operations
3. **Manual Reframing** after batch data modifications
4. **Monitor History** to track framing performance impact

**Performance Optimization:**
- Schedule batch operations during off-peak hours
- Group multiple operations before manual reframing
- Monitor lakehouse table history for optimization insights
- Balance refresh frequency with query performance needs

### 🚀 Next Steps

**Recommended Follow-up:**
- **Lab 6**: Column partitioning for enhanced performance
- **Lab 7**: High cardinality column optimization techniques
- **Lab 8**: Advanced One Lake integration patterns

**Production Implementation:**
- Establish framing policies for different data scenarios
- Implement monitoring for framing performance metrics
- Design refresh schedules based on business requirements

### 📈 Business Impact

**Direct Lake Framing enables:**
- **Real-time Analytics**: Immediate insights from operational data
- **Performance Optimization**: Efficient resource utilization
- **Scalable Architecture**: Support for growing data volumes
- **Operational Excellence**: Reliable data synchronization patterns

**ROI Indicators:**
- Reduced refresh times through optimized framing
- Improved user experience with real-time data
- Lower infrastructure costs through efficient resource usage
- Enhanced data governance through controlled synchronization